In [ ]:
import glob
import os.path as osp
import random
import numpy as np 
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import os
import urllib.request
import zipfile

import torch
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.datasets as dsets
from torchvision import models, transforms

In [ ]:
# data 폴더가 존재하지 않는 경우 작성한다
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)


In [ ]:
# ImageNet의 class_index를 다운로드한다
# Keras에서 제공하는 항목
# https://github.com/fchollet/deep-learning-models/blob/master/imagenet_utils.py

url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
save_path = os.path.join(data_dir, "imagenet_class_index.json")

if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)


In [ ]:
# 1.3절에서 사용하는 개미와 벌의 화상 데이터를 다운로드하여 압축을 해제한다
# PyTorch의 튜토리얼로 제공되는 항목
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
save_path = os.path.join(data_dir, "hymenoptera_data.zip")

if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

    # ZIP 파일을 읽는다
    zip = zipfile.ZipFile(save_path)
    zip.extractall(data_dir)  # ZIP을 압축 해제
    zip.close()  # ZIP 파일을 닫는다

    # ZIP 파일을 삭제
    os.remove(save_path)


※실시 완료 사항

골든 리트리버 이미지를 수동 다운로드

https://pixabay.com/ja/photos/goldenretriever-%E7%8A%AC-3724972/
의 640×426 사이즈 화상
(사진 권리 정보: CC0 Creative Commons, 상용 이용 무료, 저작자 표시 필요 없음)을, data 폴더 바로 아래에 배치함.

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

#### RandomResizedCrop을 지정된 scale로 이미지를 확대 및 축소를 한다
#### RandomRHorizontalFlip을 통해 좌우를 50% 확률로 반전시킨다.

데이터의 수가 적을땐 위 과정을 통해 데이터를 증폭시킨다.

In [ ]:
class ImageTransform():
    
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train' : transforms.Compose([
                transforms.RandomResizedCrop(
                resize, scale = (0.5, 1.0)), # 데이터 확장
                transforms.RandomHorizontalFlip(), # 데이터 확장
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val' : transforms.Compose([
                transforms.Resize(resize),
                transforms.CenterCrop(resize), 
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }
    
    def __call__(self, img, phase = 'train'):
        return self.data_transform[phase](img)
        

In [ ]:
image_file_path = './data/goldenretriever-3724972_960_720.jpg'
img = Image.open(image_file_path)
plt.imshow(img);

In [ ]:
size = 224 
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

transform = ImageTransform(size, mean, std)
img_transformed = transform(img, phase = 'train')

In [ ]:
# c x h x w 를 h x w x c 로 변환하고, 0~1 로 값을 제한.
img_transformed = img_transformed.transpose((1, 2, 0))
img_transformed = np.clip(img_transformed, 0, 1)
plt.imshow(img_transformed);

In [ ]:
# 개미, 벌 분류 : train 243장, val 153장
def make_datapath_list(phase = 'train'):
    rootpath = './data/hymenoptera_data/'
    target_path = osp.join(rootpath+phase+'/**/*.jpg')
    path_list = []
    
    for path in glob.glob(target_path): # 파일 경로를 가지고 온다.
        path_list.append(path)
    
    return path_list 


train_list = make_datapath_list(phase = 'train')
val_list = make_datapath_list(phase = 'val')

train_list

In [ ]:
class HymenopteraDataset(Dataset):
    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        
        img_transformed = self.transform(img, self.phase)
        
        if self.phase == 'train':
            label = img_path[30:34]
        elif self.phase == 'val':
            label = img_path[28:32]
            
        if label == 'ants':
            label = 0
            
        elif label == 'bees':
            label = 1
            
        return img_transformed, label
        

In [ ]:
train_dataset = HymenopteraDataset( file_list = train_list, transform = ImageTransform(size, mean, std),
                                  phase = 'train')

val_dataset = HymenopteraDataset(file_list = val_list, transform = ImageTransform(size, mean, std),
                                phase = 'val')

index = 0
print(train_dataset.__getitem__(index)[0].size())
print(val_dataset.__getitem__(index)[1])

In [ ]:
# 미니 배치 크기 지정
batch_size = 32

# 데이터 로더 작성
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle=True, drop_last=True)

# 사전형 변수에 정리.
dataloaders_dict = {'train' : train_dataloader, 'val' : val_dataloader}

batch_iterator = iter(dataloaders_dict['train'])
inputs, labels = next(batch_iterator) # 첫번째 요소 추출
print(inputs.size())
print(labels)
print(labels[labels == 1].size(), labels[labels == 0].size())


In [ ]:
# 학습된 VGG-16 모델 로드 
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained).to(device)

# 벌과 개미 두가지 클래스로 분류하기 때문에 마지막 fc.layer를 수정해준다.
net.classifier[6] = nn.Linear(in_features = 4096, out_features=2).to(device)

net.train()
print('네트워크 설정 완료 : 학습된 가중치를 읽어들여 훈련 모드로 설정')

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
# pretrained 된 모델의 파라미터를 고정할 때 설정.
requires_grad = False

# transfer-learning에서 학습시킬 파라미터를 저장
params_to_update = []

update_param_names = ['classifier.6.weight', 'classifier.6.bias']

for name, param in net.named_parameters():
    if name in update_param_names:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False
        
print('------------')
print(params_to_update)

In [ ]:
for name, param in net.named_parameters():
    print(name, param.size())
    break

In [ ]:
optimizer = optim.SGD(params = params_to_update, lr = 0.001, momentum = 0.9)

In [ ]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1} / {num_epochs}')
        print('-----------------')
        
        # 에폭별 학습 및 검증 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0
            
            # 학습하지 않을 시 검증 성능을 학인하기 위해 epoch = 0의 훈련 생략
            if (epoch == 0) and (phase == 'train'):
                continue
                
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    y_pred = torch.argmax(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                    
                    epoch_loss += loss.item() * inputs.size(0)
                    
                    epoch_corrects += (y_pred == labels).sum().item()
                    
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects / len(dataloaders_dict[phase].dataset)
            print(f'{phase} Loss : {epoch_loss :.4f}, Acc : {epoch_acc:.4f}')       

In [ ]:
num_epochs = 10
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs)